# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Импорт основных библиотек и данных. Изучение данных и их подготовка к анализу

### Импорт библиотек и файлов с данными

In [1]:
import pandas as pd
# Импортируем библиотеку pymystem3 для лемматизации данных о целях получения кредита
from pymystem3 import Mystem

Делаем преднастройку - ограничиваем видимость для цифр с плавающей запятой в две цифры, а также настраиваем отображение всех колонок.

In [2]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)

### Импорт и изучение данных

In [3]:
df = pd.read_csv('data.csv')

Для изучения данных будем использовать функцию inspect.

In [4]:
def inspect(df):
    display(df.sample(10, random_state=1)) 
    df.info() 
    print ('Количество дубликатов -', df.duplicated().sum())
    numeric_columns=df.describe().columns
    display(df.describe())
    display()
    df.columns = [x.lower().replace(' ', '_') for x in df.columns.values]
    for column_name in df.columns:
        if 'datetime' in column_name:
            df[column_name] = pd.to_datetime(df[column_name])
        else:
            display(df[column_name].value_counts())
            print(50*'*')
    return df

In [5]:
# Дополнительно посмотрим на данные таблицы.
inspect(df)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1383,0,353802.81,37,среднее,1,вдовец / вдова,2,F,пенсионер,0,216452.23,строительство недвижимости
300,1,-359.19,33,СРЕДНЕЕ,1,гражданский брак,1,M,сотрудник,0,223001.62,на проведение свадьбы
6565,2,-1064.85,35,среднее,1,гражданский брак,1,F,компаньон,0,163591.21,свадьба
17027,0,NaN,48,высшее,0,гражданский брак,1,F,сотрудник,0,NaN,операции с жильем
4077,0,-7059.10,45,высшее,0,гражданский брак,1,F,компаньон,1,194820.19,сыграть свадьбу
10437,0,-2258.30,50,среднее,1,женат / замужем,0,F,сотрудник,0,146768.25,операции с коммерческой недвижимостью
9631,1,-1045.75,23,среднее,1,женат / замужем,0,M,сотрудник,0,81734.57,операции с коммерческой недвижимостью
19867,0,NaN,39,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,дополнительное образование
9124,0,-282.19,48,среднее,1,в разводе,3,F,компаньон,0,284275.68,строительство жилой недвижимости
19634,0,-1320.10,46,среднее,1,женат / замужем,0,F,компаньон,0,184569.03,автомобили


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
Количество дубликатов - 54


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.00,19351.00,21525.00,21525.00,21525.00,21525.00,19351.00
mean,0.54,63046.50,43.29,0.82,0.97,0.08,167422.30
std,1.38,140827.31,12.57,0.55,1.42,0.27,102971.57
min,-1.00,-18388.95,0.00,0.00,0.00,0.00,20667.26
25%,0.00,-2747.42,33.00,1.00,0.00,0.00,103053.15
50%,0.00,-1203.37,42.00,1.00,0.00,0.00,145017.94
75%,1.00,-291.10,53.00,1.00,1.00,0.00,203435.07
max,20.00,401755.40,75.00,4.00,4.00,1.00,2265604.03


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

**************************************************


...

253875.64    1
157691.85    1
70113.90     1
116196.52    1
157205.83    1
            ..
168880.59    1
148042.72    1
60039.33     1
175979.76    1
82047.42     1
Name: total_income, Length: 19351, dtype: int64

**************************************************


свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**************************************************


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,1,-4024.80,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля
2,0,-5623.42,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья
3,3,-4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование
4,0,340266.07,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.32,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.86,операции с жильем
21521,0,343937.40,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.81,сделка с автомобилем
21522,1,-2113.35,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.56,недвижимость
21523,3,-3112.48,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.05,на покупку своего автомобиля


**Вывод**

В таблице 21525 строк.  
В столбцах days_employed и total_income есть пропуски. Пропуски более, чем у 10% данных. Удаление этих данных может повлиять на итоги исследования. Поэтому пропуски будут заполняться вручную. 


Столбцы с категориальными переменными: education, education_id, family_status, family_status_id, gender, income_type, debt, purpose.  
Столбцы с количественными переменными: children, days_employed, dob_years, total_income.

Пропуски данных в столбцах days_employed (общий трудовой стаж в днях) и в total_income (ежемесячный доход). Так как значения переменных этих столбцов количественные, пропуски будут заполняться характерными значениями.  

Просмотр данных таблицы позволил увидеть ошибки заполнения данных. Столбец days_employed имеет отрицательные значения.  
По первым 20 строкам можно предположить, что отрицательные значения могут быть связаны со столбцом income_type. Значения отрицательные у всех категорий заемщиков, кроме пенсионеров. Возможно, знак минус имеет смыслоразличительное значение в данном случае.   

В данных 54 дубликата.  

Так как  для решения поставленной задачи стаж работы заемщиков не требуется, указанные выше гипотезы в данный момент проверяться не будут. Но заполним и эти пропуски данными для будущих исследований.  

## Предобработка данных

### Обработка пропусков

In [6]:
# Проверим строки с пропусками в столбце days_employed
df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


После проверки пустых значений в столбце days_employed появилось предположение, что пропуски в столбцах total_income и days_employed будут у одних и тех же строк.  
Проверим это.

In [7]:
df[(df['days_employed'].isna()) & (df['total_income'].isna())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Предположение оказалось верным. Всего, по полученным в самом начале данным, пропусков 2174, в полученном массиве их также 2174.  
Заполним пропуски в столбце трудового стажа (столбец days_employed).  
Медиана будет использоваться для того, чтобы избежать неточностей метода расчета среднего арифметического
в ситуациях, когда в данных есть слишком маленькие или слишком большие значения, которые могут повлиять на результат.

In [8]:
# Создаем массив с индексами строк с пустыми значениями для замены значений циклом 
df_nan_rows = df[df['days_employed'].isna()].index

# Создаем массив с уникальными значениями возраста для заполнения пропущенных значений возраста циклом.
unique_dob_years = df['dob_years'].unique()

# Заполняем пропуски (решил делать циклом, а не через .fillna(), т.к. при замене пропусков в столбце с доходом 
#цикл будет нагладнее, хоть и занимать больше времени для обработки):
for row in df_nan_rows:
    for year in unique_dob_years:
        if df.loc[row, 'dob_years'] == year:
            df.loc[row, 'days_employed'] = df[df['dob_years'] == year]['days_employed'].median()

# Проверяем массив на наличие пустых значений в столбце days_employed:
df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Пустых значений не осталось.   

Недостающую информацию о доходах будем также заполнять медианными значениями.  
Расчет медианных значений для определения дохода каждого конкретного заемщика будем брать исходя из количества детей, образования и типа занятости.

In [9]:
# Делаем массивы для уникальных значений кол-ва детей, образования и типа занятости:
df_children_unique = df['children'].unique()
df_education_unique = df['education_id'].unique()
df_income_type_unique = df['income_type'].unique()

# Циклом осуществляем перебор значений:
for row in df_nan_rows:
    for children_unique in df_children_unique:
        for education_unique in df_education_unique: 
            for income_type_unique in df_income_type_unique: 
                if (df.loc[row, 'children'] == children_unique) & (df.loc[row, 'education_id'] == education_unique) & (df.loc[row, 'income_type'] == income_type_unique):
                    df.loc[row, 'total_income'] = df[(df['children'] == children_unique) & 
                                                     (df['education_id'] == education_unique) & 
                                                     (df['income_type'] == income_type_unique)]['total_income'].median()

# Проверим наличие нулевых значений через .info():
df.info()

# Продумать альтернативу через apply

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Все пропуски заполнены.

**Вывод**

Отсутствие данных в столбцах days_employed и total_income связано между собой. Данные о ежемесячном доходе отсутствуют по тем строкам, по которым отсутствует общий трудовой стаж.  
Возможными причинами отсутствия данных могут быть: ошибки при вводе данных, отсутствие официального стажа работы у конкретных заемщиков (и, соответственно, официального ежемесячного дохода).  
Также предполагаю, что если ведется дополнительная проверка стажа и дохода со стороны кредитного отдела банка, то отсутствие данных может являться следствием того, что предоставленные заемщиками данные не прошли проверку.  

Данные о стаже были заполнены медианными значениями для каждого возраста отдельно.  
Данные о ежемесячном доходе были заполнены медианными значениями на основании имеющейся статистики в таблице с учетом количества детей у заемщиков, уровня образования и типа занятости.


### Замена типа данных
Заменим тип данных у столбцов days_employed и total_income с float на int (данные этих столбцов не требуют детализации до долей, целого остатка будет достаточно для проведения анализа).  

In [10]:
df = df.astype({'days_employed': 'int32', 'total_income': 'int32'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


**Вывод**

Мы заменили тип данных у столбцов days_employed и total_income с float на int, т.к. данные этих столбцов
не требуют детализации до долей, целого остатка будет достаточно для проведения анализа.  
Столбцы education, family_status, gender, income_type и purpose хранят текстовые данные. Для них тип данных object подходит.  
Остальные стобцы имеют тип данных int, что также нам подходит.

### Обработка дубликатов
Всего 54 задублированные строки. Т.к. это меньше половины доли процента, удаление этих данных не повлияет на итоги исследования. 

In [11]:
# Проверим кол-во полных дубликатов в таблице:
df.duplicated().sum()

54

In [12]:
# Удаляем строки-дубли:
df = df.drop_duplicates().reset_index(drop = True)

# Перепроверяем кол-во полных дубликатов в таблице:
df.duplicated().sum()

0

Строк-дублей в таблице больше нет.

In [13]:
# Проверим дубли в столбце education:
df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

В столбце много неявных дублей, которые можно устранить посредством приведения всех букв к единому регистру.

In [14]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Перепроверим данные на дубликаты после приведения значений к нижнему регистру. 

In [15]:
df.duplicated().sum()

17

Появилось 17 дубликатов. Удалим полученные дубликаты.

In [16]:
df = df.drop_duplicates().reset_index(drop = True)

In [17]:
# Проверим дубли в столбце family_status:
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

В этом столбце дубликатов нет.

In [18]:
# Проверим дубли в столбце income_type:
df['income_type'].value_counts()

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Дубликатов в этом стоблце нет. Можно обратить внимание на статусы "сотрудник", "компаньон" и "предприниматель".  
Но без дополнительной консультации с заказчиком для самостоятельного принятия решения не хватает информации. 

**Вывод**

В таблице было 54 задублированные строки. Мы удалили строки-дубли и обновили индексацию строк.  
Дубли строк могли появиться в случае ошибки ввода, или в случае повторного обращение того же заемщика в кредитный отдел банка.  

Далее мы проверили  неявные дубликаты в столбцах education, family_status и income_type. Устранили дубликаты в столбце education, связанные с разницей регистров букв. Эти ошибки связаны, скорее всего, с тем, что данное поле заполняется сотрудниками вручную, и нет маски ввода/ограничений по вводу данных в поле.  

### Лемматизация

In [ ]:
m = Mystem() 

# Лемматизируем данные, добавляем их в отдельную таблицу tags:
tags = []
for i in range(len(df['purpose'])):
    tags.append(m.lemmatize(df.loc[i, 'purpose']))

# Добавляем леммитизированные данные в новый столбец purpose_tags таблицы tags:    
df['purpose_tags'] = pd.Series(tags)

# Проверяем таблицу с новым столбцом
df

**Вывод**  

Мы лемматизировали (привели все слова столбца purpose к их леммам - к их словарным формам) информацию о целях получения кредита для упрощения последующего анализа влияния цели кредита на своевременный возврат кредита в срок. 

### Категоризация данных
Вынесем уровень образования и семейный статус в отдельные словари для удобства дальнейшего обращения к данным таблиц.  

In [ ]:
# Сделаем словарь для уровня образования:
education_dict = df[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop = True)
education_dict

In [ ]:
# Делаем аналогичный словарь для семейного статуса:
family_status_dict = df[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop = True)
family_status_dict

Категоризируем данные столбца purpose_tags для дальнейшего анализа.

In [ ]:
# Для этого сначала посмотрим, какие уникальные значения встречаются в этом столбце:

df['purpose_tags'].value_counts()

Все цели можно поделить на следующие категории: 1) автомобиль; 2) свадьба; 3) недвижимость/жилье; 4) образование.  
Заменим данные типа Series в столбце purpose_tags на выделенные выше категории.   
Для этого напишем функцию, которая пройдет по всем строкам массива, и возвратит нужную категорию, если она присутствует в строке. Добавим в функцию условие try/except для избежания ошибок при отработке кода:  


In [ ]:
def categorize(data):
    for i in range(len(data['purpose_tags'])):
        try:
            if 'автомобиль' in data.loc[i, 'purpose_tags']:
                data.loc[i, 'purpose_tags'] = 'автомобиль'
            elif 'свадьба' in data.loc[i, 'purpose_tags']:
                data.loc[i, 'purpose_tags'] = 'свадьба'
            elif 'недвижимость' in data.loc[i, 'purpose_tags']:
                data.loc[i, 'purpose_tags'] = 'недвижимость'
            elif 'жилье' in data.loc[i, 'purpose_tags']:
                data.loc[i, 'purpose_tags'] = 'недвижимость'
            elif 'образование' in data.loc[i, 'purpose_tags']:
                data.loc[i, 'purpose_tags'] = 'образование'
        except:
            data.loc[i, 'purpose_tags'] = 'error'

# Так как жилье и недвижимость синонимичные понятия, в обоих случаях будем присваивать категорию 'недвижимость'.
# Применим функцию к таблице, заменим значение столбца purpose_tags на категории:

categorize(df)


In [ ]:
# Переименуем столбец purpose_tags в category и проверим таблицу на предмет наличие категории error:
df.rename(columns={'purpose_tags': 'category'}, inplace=True)
df['category'].value_counts()

**Вывод**  

Были созданы словари уровня образования и семейного статуса заемщиков для дальнейшего упрощения работы с данными таблицы.  

Выделенные в рамках лемматизации словарные формы слов были категоризированы на четыре группы.
Основные причины, по которым заемщики брали кредит: 1) автомобиль; 2) свадьба; 3) недвижимость; 4) образование. 
Эти категории были добавлены в таблицу в столбец 'category'.

## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
# Проверим количество уникальных значений в столбце children: 
df['children'].value_counts()

В данных есть артефакты: -1 ребенок и 20 детей. Всего этих значений 123. Это чуть больше 0,57% от общего количества данных.  
Можно предположить, что "-1" - это написанное с ошибкой "1", а "20" - это "2". У нас нет возможности проконсультироваться с заказчиком для определения точной причины этих ошибок.  

Так как количество этих данных мал в масштабе массива данных, было бы целесообразно удалить эти данные.  
Но в связи с тем, что нам потребуется рассчитать не только зависимость между наличием детей и возвратом кредита в срок, 
оставим эти данные, скорректировав данные о количестве детей на "1" и "2".

In [ ]:
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

# Проверяем обновленные данные по столбцу children:
df['children'].value_counts()

Проверим количество долгов в разрезе количества детей:

In [ ]:
df.pivot_table(index=['children'], values='debt', aggfunc='sum')
print(df.pivot_table(index=['children'], values='debt', aggfunc='sum')) 

Выведем процент заемщиков с наличинем задолженностей по возврату кретитов в разрезе количества детей, чтобы ответить на поставленный вопрос. 

In [ ]:
# Выведем таблицу через display(), отсортируем ее по возрастанию доли задолженности,
#и перемиенуем столбец с итогами расчетов для большей наглядности:
df_child_pivot = df.pivot_table(index=['children'], values='debt', aggfunc='mean').sort_values('debt')
df_child_pivot.rename(columns={'debt': 'Доля заемщиков с задолженностью'}, inplace=True)
display(df_child_pivot) 

**Вывод**

Согласно предоставленным заказчиком данным, определенная зависимость между наличием детей и вовзвратом кредита в срок наблюдается, хоть корреляция и не сильно выраженная.  
Люди, не имеющие детей, чаще возвращают кредит в срок, нежели чем те, кто имеет от одного до четырех детей (ок. 7,5% заемщиков без детей имели ранее задолженности по кредитам).  
Среди тех заемщиков, у кого от одного до четырех детей, наименьшее количество задолженностей по возвратам кредитов имели те, у кого три ребенка (ок. 8,2% заемщиков имели задолженности). Далее идут заемщики с одним ребенком (~9,5%). Наибольшее количество задолженностей по кредитам имели заемщики с четыремя детьми - 9,7% заемщиков.  
У людей с 5 детьми не имелось задолженностей по возврату кредитов вообще. Однако данная выборка нерепрезентативна - в таблице предоставлена информация о 9 заемщиках с пятью детьми; данных недостаточно, чтобы экстраполировать выводы для построения модели кредитного скроллинга.

Результаты полученного среза могут быть связаны с тем, что наличие детей налагает на человека больше обязательств (в т.ч. финансовых), а также предполагает больше непредсказуемых трат.  
Неожиданно меньше задолженностей по кредитам у людей с тремя детьми. Возможно, это связано с тем, что люди, имеющие трех детей, более взвешенно принимали решении о рождении третьего ребенка (имея понимание о расходах и обязательствах). Допускаю, своего рода, "эффект масштаба".

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
print(df['family_status'].value_counts())

In [ ]:
df_fam_stat_pivot = df.pivot_table(index=['family_status'], values='debt', aggfunc='mean').sort_values('debt')
df_fam_stat_pivot.rename(columns={'debt': 'Доля заемщиков с задолженностью'}, inplace=True)
display(df_fam_stat_pivot) 

**Вывод**

Зависимость между семейным положением и возвратом кредита в срок также прослеживается. В данном случае более явная, нежели чем в первом случае.  
Чаще всего в срок возвращали кредит те заемщики, которые утратили своего партнера (ок. 6,6% заемщиков имели просроченные кредиты). Больше просроченных задолженностей имели люди в разводе и в официальном браке (7,1% и 7,5% просроченных кредитов).  
Наибольшее количество задолженностей по кредитам имели люди в граджанском браке и холостые люди (9,3% и 9,7%).  

Наличие официальных отношений (даже бывших) может говорить о том, что человек более ответственно относится к возврату кредита в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для корректного анализа зависимости между уровнем дохода и возвратом кредита в срок потребуется категоризировать уровни дохода.  
Разобьем данные на следующие категории: 1) до 100000; 2) от 100000 до 150000; 3) от 150000 до 200000; 4) более 200000 (категории были выбраны в соответствии с количеством данных в таблице).  
Добавим категории в отдельный столбец income_category.

In [ ]:
for i in range(len(df['total_income'])):
    if df.loc[i, 'total_income'] < 100000:
        df.loc[i, 'income_category'] = 'до 100000'
    elif 100000 <= df.loc[i, 'total_income'] < 150000:
        df.loc[i, 'income_category'] = 'от 100000 до 150000'
    elif 150000 <= df.loc[i, 'total_income'] < 200000:
        df.loc[i, 'income_category'] = 'от 150000 до 200000'
    else: df.loc[i, 'income_category'] = 'более 200000'
  
    

In [ ]:
df['income_category'].value_counts()  

In [ ]:
df_income_pivot = df.pivot_table(index=['income_category'], values='debt', aggfunc='mean').sort_values('debt')
df_income_pivot.rename(columns={'debt': 'Доля заемщиков с задолженностью'}, inplace=True)
display(df_income_pivot) 

**Вывод**

Все заемщики были разбиты на 4 категории в зависимости от уровня дохода: 1) до 100000; 2) от 100000 до 150000; 3) от 150000 до 200000; 4) более 200000.
Зависимость между возвратом кредитов в срок и доходом таккже прослеживается. 
Меньше всего задолженностей по кредитам у людей с достатком более 200000 руб. (почти 7%). Далее идет категория с доходом до 100000 руб. (7,9% заемщиков с долгами по кредитам).  
У остальных двух категорий 8,5% и почти 8,8% заемщиков имели задолженность за кредиты. Наибольшее количество задолженностей у категории людей с доходом от 100000 до 150000 руб.  

Полагаю, что у людей с доходом более 200000 руб. есть больше возможностей для своевременного возврата кредита.  
Люди с доходом до 100000 руб. чуть более бережно обходятся со своими финансками, чем остальные заемщики (с доходом до 200000 руб.).

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
df_purpose_pivot = df.pivot_table(index=['category'], values='debt', aggfunc='mean').sort_values('debt')
df_purpose_pivot.rename(columns={'debt': 'Доля заемщиков с задолженностью'}, inplace=True)
display(df_purpose_pivot) 

**Вывод**

Цели кредита также влияют на своевременный возврат кредита в срок. 
Те, кто берет кредит на недвижимость и на свадьбу, чаще не имеют задолженностей по кредитам, нежели чем те, кто берет кредит на образование или для покупки автомобиля. 
Наименьшая доля задолженностей по кредитам среди тех, кто берет кредит на покупку недвижимости (7,2%). Кредит на покупку недвижимости важный и отвественный шаг, поэтому человек, который планирует брать кредит на эти нужды, заранее заботится о своей кредитной истории.
Следующая категория - заемщики, которые берут кредиты на свадьбу (ок. 8% заемщиков имеют задолженности по кредитам).
Наибольшее количество задолженностей по кредитам - кредиты, получаемые на образование и на приобретение автомобия (9,2% и 9,3%).  

## Общий вывод

При построении модели кредитного скоринга желательно учесть наличие детей у заемщика, его семейное положение, уровень дохода и цели кредита. Разница в доле заемщиков, которые не вернут кредит в срок, между категориями каждого среза может составлять до 3%.  
Идеальный портрет заемщика: вдова без детей с доходом от 200000 рублей, которая приобретает в кредит недвижимость. 